In [22]:
URI = "gs://tfl-cycle-1413//app/airflow/data/20230612.parquet"
URI_b = "gs://tfl-cycle-1413//app/airflow/data/20220907.parquet"


In [23]:
import pandas as pd
df = pd.read_parquet(URI_b)


In [24]:
df.head(2)


,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,124948506,420,17902,07/09/2022 00:07,424,"Ebury Bridge, Pimlico",07/09/2022 00:00,177,"Ashley Place, Victoria"
1,124948505,900,14422,07/09/2022 00:15,469,"Lindfield Street, Poplar",07/09/2022 00:00,282,"Royal London Hospital, Whitechapel"


In [25]:
df = df.rename(columns={'Rental Id': 'Number',
                                    'Start Date': 'Start date',
                                    'End Date': 'End date',
                                    'StartStation Id':'Start station number',
                                    'StartStation Name': 'Start station',
                                    'EndStation Id': 'End station number',
                                    'EndStation Name': 'End station',
                                    'Duration': 'Total duration',
                                    'Bike Id': 'Bike number'
                                    })


In [26]:
df.head(2)


,Number,Total duration,Bike number,End date,End station number,End station,Start date,Start station number,Start station
0,124948506,420,17902,07/09/2022 00:07,424,"Ebury Bridge, Pimlico",07/09/2022 00:00,177,"Ashley Place, Victoria"
1,124948505,900,14422,07/09/2022 00:15,469,"Lindfield Street, Poplar",07/09/2022 00:00,282,"Royal London Hospital, Whitechapel"


In [27]:
df['Total duration'] = df['Total duration'].map(lambda x: x*0.001)


In [29]:
df = df.round({'Total duration': 1})
df.head(2)


,Number,Total duration,Bike number,End date,End station number,End station,Start date,Start station number,Start station
0,124948506,0.4,17902,07/09/2022 00:07,424,"Ebury Bridge, Pimlico",07/09/2022 00:00,177,"Ashley Place, Victoria"
1,124948505,0.9,14422,07/09/2022 00:15,469,"Lindfield Street, Poplar",07/09/2022 00:00,282,"Royal London Hospital, Whitechapel"


In [7]:
df["date"] = "current_date"
df.head(2)


,Number,Total duration,Bike number,End date,End station number,End station,Start date,Start station number,Start station,date
0,124948506,0.42,17902,07/09/2022 00:07,424,"Ebury Bridge, Pimlico",07/09/2022 00:00,177,"Ashley Place, Victoria",current_date
1,124948505,0.90,14422,07/09/2022 00:15,469,"Lindfield Street, Poplar",07/09/2022 00:00,282,"Royal London Hospital, Whitechapel",current_date


In [34]:
from google.cloud import storage

def get_all_files_dates(bucket_name):
    storage_client = storage.Client.from_service_account_json('/home/d.bodin03/.gcp_keys/le-wagon-de-bootcamp.json')
    bucket = storage_client.get_bucket(bucket_name)

    iterator = bucket.list_blobs(prefix='bronze/')
    response = iterator._get_next_page_response()
    return [i['name'].split("/")[-1].startwith("20150301") for i in response['items']]
data = get_all_files_dates("tfl-cycle-1413")


AttributeError: 'str' object has no attribute 'startwith'

In [61]:
import os
def get_wanted_files_dates(bucket_name:str, yyyymm:str):
    storage_client = storage.Client.from_service_account_json('/home/d.bodin03/.gcp_keys/le-wagon-de-bootcamp.json')
    bucket = storage_client.get_bucket(bucket_name)
    iterator = bucket.list_blobs(prefix='bronze/')
    response = iterator._get_next_page_response()

    get_file_names = [i['name'].split("/")[-1] for i in response['items']]
    get_dates = [i[:7] for i in get_file_names]

    file_list = []
    for f_date, f_names in zip(get_dates, get_file_names) :
        if f_date.startswith(yyyymm):
            file_list.append(f'./data/{f_names}')

    return file_list


In [78]:
l = get_wanted_files_dates("tfl-cycle-1413", "202108")


In [79]:
l


['./data/20210804.parquet',
 './data/20210811.parquet',
 './data/20210818.parquet',
 './data/20210825.parquet']

In [76]:
get_wanted_objects = [f'bronze/{f.split("/")[-1]}' for f in l]


In [77]:
get_wanted_objects


['bronze/20210804.parquet',
 'bronze/20210811.parquet',
 'bronze/20210818.parquet',
 'bronze/20210825.parquet']

In [72]:
def prepare_data(file_path:str, current_date:str):
    df = pd.read_parquet(file_path)

    if "Rental Id" in df.columns :
        df = df.rename(columns={'Rental Id': 'Number',
                                    'Start Date': 'Start date',
                                    'End Date': 'End date',
                                    'StartStation Id':'Start station number',
                                    'StartStation Name': 'Start station',
                                    'EndStation Id': 'End station number',
                                    'EndStation Name': 'End station',
                                    'Duration': 'Total duration',
                                    'Bike Id': 'Bike number'
                                    })
    else:
        df = df.drop(['Bike model','Total duration'], axis=1)
        df = df.rename(columns={"Total duration (ms)":"Total duration"})
        df['Total duration'] = df['Total duration'].map(lambda x: x*0.001)
        df = df.round({'Total duration':0})

    df["date"] = current_date

    df.to_parquet(f'./data/silver/{file_path.split("/")[-1]}')


PermissionError: [Errno 13] Permission denied: './data/silver/20210825.parquet'

In [33]:
data


['20150104.parquet',
 '20150118.parquet',
 '20150201.parquet',
 '20150215.parquet',
 '20150301.parquet',
 '20150316.parquet',
 '20150401.parquet',
 '20150417.parquet',
 '20150503.parquet',
 '20150517.parquet',
 '20150531.parquet',
 '20150613.parquet',
 '20150628.parquet',
 '20150712.parquet',
 '20150726.parquet',
 '20150808.parquet',
 '20150823.parquet',
 '20150906.parquet',
 '20150920.parquet',
 '20151004.parquet',
 '20151018.parquet',
 '20151101.parquet',
 '20151115.parquet',
 '20151128.parquet',
 '20151213.parquet',
 '20151225.parquet',
 '20160110.parquet',
 '20160124.parquet',
 '20160207.parquet',
 '20160221.parquet',
 '20160306.parquet',
 '20160401.parquet',
 '20160501.parquet',
 '20160518.parquet',
 '20160525.parquet',
 '20160601.parquet',
 '20160608.parquet',
 '20160615.parquet',
 '20160622.parquet',
 '20160629.parquet',
 '20160706.parquet',
 '20160713.parquet',
 '20160720.parquet',
 '20160727.parquet',
 '20160803.parquet',
 '20160810.parquet',
 '20160817.parquet',
 '20160824.pa

In [4]:
keys = [d for d in get_all_files_dates("tfl-cycle-1413")]
keys


['20150104',
 '20150118',
 '20150201',
 '20150215',
 '20150301',
 '20150316',
 '20150401',
 '20150417',
 '20150503',
 '20150517',
 '20150531',
 '20150613',
 '20150628',
 '20150712',
 '20150726',
 '20150808',
 '20150823',
 '20150906',
 '20150920',
 '20151004',
 '20151018',
 '20151101',
 '20151115',
 '20151128',
 '20151213',
 '20151225',
 '20160110',
 '20160124',
 '20160207',
 '20160221',
 '20160306',
 '20160401',
 '20160501',
 '20160518',
 '20160525',
 '20160601',
 '20160608',
 '20160615',
 '20160622',
 '20160629',
 '20160706',
 '20160713',
 '20160720',
 '20160727',
 '20160803',
 '20160810',
 '20160817',
 '20160824',
 '20160831',
 '20160907',
 '20160914',
 '20160921',
 '20160928',
 '20161005',
 '20161012',
 '20161019',
 '20161026',
 '20161102',
 '20161109',
 '20161116',
 '20161123',
 '20161130',
 '20161207',
 '20161214',
 '20161221',
 '20161228',
 '20170104',
 '20170111',
 '20170118',
 '20170125',
 '20170201',
 '20170208',
 '20170215',
 '20170222',
 '20170301',
 '20170308',
 '20170322',

In [21]:
!pwd


/home/d.bodin03/code/DanyBodin/tfl-cycle


In [4]:
df.columns


Index(['Number', 'Start date', 'Start station number', 'Start station',
       'End date', 'End station number', 'End station', 'Bike number',
       'Bike model', 'Total duration', 'Total duration (ms)'],
      dtype='object')

In [8]:
df['Total duration (ms)'] = df['Total duration (ms)'].map(lambda x : x*0.001)

df = df.rename(columns={"Total duration (ms)":"Total duration"})
df.head()


,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration
0,131615942,2023-06-18 23:59,1106,"Rampayne Street, Pimlico",2023-06-19 00:03,1087,"Belgrave Road, Victoria",56222,CLASSIC,4m 16s,2.563710e-07
1,131615943,2023-06-18 23:59,1204,"Queen's Gate (North), Kensington",2023-06-19 00:15,200166,"Southerton Road, Hammersmith",59682,CLASSIC,15m 23s,9.239680e-07
2,131615944,2023-06-19 00:00,22179,"Exhibition Road, Knightsbridge",2023-06-19 00:13,200120,"Grant Road East, Clapham Junction",55015,CLASSIC,13m 19s,7.992960e-07
3,131615939,2023-06-18 23:58,3504,"Moor Street, Soho",2023-06-19 00:55,1150,"Queensway, Kensington Gardens",41885,CLASSIC,56m 59s,3.419163e-06
4,131615940,2023-06-18 23:58,300082,"Bridge Avenue, Hammersmith",2023-06-19 00:04,200181,"Richmond Way, Shepherd's Bush",40935,CLASSIC,5m 27s,3.270400e-07
